# Problema 1

In [11]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
torch.pi = torch.tensor(np.pi)
from torch.distributions import multivariate_normal

def evaluate_gaussian_2d(mu_array, Sigma, x_array):
    """Calcula la función Gaussiana multi-variable.

    Args:
        x (torch.Tensor): Tensor de tamaño (N, 2) que representa un conjunto de N puntos en R^2.
        mu (torch.Tensor): Tensor de tamaño (2,) que representa el vector de medias de la Gaussiana.
        sigma (torch.Tensor): Tensor de tamaño (2, 2) que representa la matriz de covarianza de la Gaussiana.

    Returns:
        torch.Tensor: Tensor de tamaño (N,) que representa los valores de la función Gaussiana evaluados en cada punto de x.
    """
    num_dimensions = mu_array.shape[0]
    print("num dimensions ", num_dimensions)
    #determinant_Sigma = torch.det(Sigma)
    #normalization factor
    norm_factor =  1 / torch.sqrt((2 * torch.pi)** num_dimensions)
    #cuadratic form exponent
    cuadratic_form = (x_array - mu_array).transpose(0, 1).mm(Sigma.inverse()).mm(x_array - mu_array)
    #final value
    gauss_result = norm_factor * torch.exp(-0.5*cuadratic_form)
    return gauss_result

# Crea una cuadrícula de 100x100 puntos en R^2
x = torch.linspace(0, 40, 100, dtype=torch.float32)
y = torch.linspace(0, 50, 100, dtype=torch.float32)
X, Y = torch.meshgrid(x, y)
X_tensor = torch.stack([X.flatten(), Y.flatten()], axis=1)


# Define los parámetros de la función Gaussiana
mu = torch.tensor([20, 30], dtype=torch.float32)
sigma = torch.tensor([[2, 0], [0, 10]], dtype=torch.float32)

# Evalúa la función Gaussiana en cada punto de la cuadrícula
Z_tensor = evaluate_gaussian_2d(mu, sigma, X_tensor)

# Convierte el tensor de PyTorch en una matriz de NumPy de tamaño (100, 100)
Z = Z_tensor.reshape(100, 100).detach().numpy()

# Grafica la superficie de la función Gaussiana
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none')
ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.set_zlabel('f(x)')
plt.show()

# Grafica las curvas de nivel de la función Gaussiana
fig = plt.figure()
plt.contour(X, Y, Z, cmap='viridis')
plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

num dimensions  2


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x10000 and 2x2)

### Pregunta 3

In [27]:
def createData(numberSamplesPerClass = 2, mean1 = [2, 2], mean2 = [26, 26], cov_matrix = torch.tensor([[1, 0], [0, 1]])):
    """
    Creates the data to be used for training, using a GMM distribution
    @param numberSamplesPerClass, the number of samples per class
    @param mean1, means for samples from the class 1
    @param mean2, means for samples from the class 2
    @param stds1, standard deviation for samples, class 1
    @param stds2, standard deviation for samples, class 2    """
    means = torch.zeros(2)
    # Ones to concatenate for bias
    ones = torch.ones(numberSamplesPerClass, 1)
    means[0] = mean1[0]
    means[1] = mean1[1]
    # Covariance matrix creation with identity
    covarianceMatrix = torch.eye(2)
    covarianceMatrix[0, 0] = cov_matrix[0, 0]
    covarianceMatrix[0, 1] = cov_matrix[0, 1]
    covarianceMatrix[1, 0] = cov_matrix[1, 0]
    covarianceMatrix[1, 1] = cov_matrix[1, 1]
    samplesClass1 = createDataOneClass(means, covarianceMatrix, numberSamplesPerClass)
    means[0] = mean2[0]
    means[1] = mean2[1]
    samplesClass2 = createDataOneClass(means, covarianceMatrix, numberSamplesPerClass)
    # Concatenates the ones for the bias
    samplesClass1Bias = torch.cat((ones, samplesClass1), 1)
    samplesClass2Bias = torch.cat((ones, samplesClass2), 1)
    samplesAll = torch.cat((samplesClass1, samplesClass2), 0)
    plt.scatter(samplesClass1[:, 0], samplesClass1[:, 1])
    #plt.scatter(samplesClass2[:, 0], samplesClass2[:, 1], marker = 'x')
    plt.show()
    #Create samples without bias
    samplesAll = torch.cat((samplesClass1, samplesClass2), 0)
    
    #Create targets
    targetsClass1 = torch.ones(numberSamplesPerClass, 1)
    targetsClass2 = torch.zeros(numberSamplesPerClass, 1)
    targetsAll = torch.cat((targetsClass1, targetsClass2), 0)
    
    return (targetsAll, samplesAll)


'''
Creates data with gaussian distribution
'''
def createDataOneClass(means, covarianceMatrix, numberSamples):
    # Inits the bi gaussian data generator
    multiGaussGenerator = multivariate_normal.MultivariateNormal(means, covarianceMatrix)
    # Takes the samples
    samples = multiGaussGenerator.sample(torch.Size([numberSamples]))
    
    return samples

In [26]:
(TargetsAll, SamplesAll) = createData(numberSamplesPerClass = 10000, mean1 = [12,12], mean2 = [20, 20], stds1 = [3, 3], stds2 = [2, 2])   